<a href="https://colab.research.google.com/github/whkwls2653/Emotion-Recognition/blob/main/tip_colab_%ED%8C%8C%EC%9D%BC%EC%A0%80%EC%9E%A5%EB%B0%8F%EC%97%85%EB%A1%9C%EB%93%9C_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torc
##Kobert pretrained model
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
##numpy version for error
!pip install numpy==1.23.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-2_a272h0
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-2_a272h0
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
Looking in indexes: http

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
print(np.__version__)
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook

1.23.1


In [4]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [7]:
sess1_f.sample(n=10)


,Numb,Wav,Unnamed: 2,ECG,Unnamed: 4,E4-EDA,Unnamed: 6,E4-TEMP,Unnamed: 8,Segment ID,...,Unnamed: 33,Eval08F,Unnamed: 35,Unnamed: 36,Eval09M,Unnamed: 38,Unnamed: 39,Eval10F,Unnamed: 41,Unnamed: 42
238,238.0,23.282,35.125,0.1265266,-0.3795799,0.211191,0.200951,29.37,29.37,Sess01_script06_F002,...,3,disgust,2,3,disgust,2,3,disgust,2,3
326,326.0,32.427,33.361,0.02530533,-0.06507084,0.302552,0.287181,29.25,29.25,Sess01_impro02_F003,...,3,sad,2,2,neutral,3,3,neutral,4,3
24,24.0,181.221,182.593,-0.05061065,0.07230093,0.477419,0.494058,30.79,30.79,Sess01_script01_F010,...,3,disgust,1,5,neutral,2,4,fear,1,3
414,414.0,252.639,265.3,0.05061065,-0.2530532,NaN,NaN,NaN,NaN,Sess01_impro03_F024,...,4,fear,2,2,surprise,2,4,fear,1,4
306,306.0,200.171,202.758,-0.03253542,0.003615046,1.267214,1.253124,29.85,29.85,Sess01_impro01_F009,...,4,angry,2,2,neutral,2,4,angry,2,3
16,16.0,112.794,132.914,0.06507084,0.08314607,0.45822,0.531177,30.75,30.77,Sess01_script01_M010,...,4,fear,1,4,neutral,2,4,neutral,1,3
476,476.0,259.624,261.876,-0.1843674,-0.1482169,NaN,NaN,NaN,NaN,Sess01_impro04_M032,...,4,happy,4,4,neutral,4,3,neutral,4,3
450,450.0,165.764,170.854,-0.01084514,0.02892037,NaN,NaN,NaN,NaN,Sess01_impro04_M019,...,4,neutral,4,3,neutral,4,3,neutral,4,3
309,309.0,216.823,225.343,0,-0.01446019,1.223664,1.209575,29.89,29.89,Sess01_impro01_M010,...,4,angry,2,2,angry,2,3,angry,2,3
393,393.0,127.093,142.6,0.03615047,-0.151832,0.280777,NaN,29.37,NaN,Sess01_impro03_M012,...,3,neutral,2,2,neutral,2,3,neutral,1,3


In [101]:
import pandas as pd
import os

annotation_root= '/content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/annotation'

allfile_datalist=[]
ambig_annotation=0
no_file=0
for i, annotation in enumerate(os.listdir(annotation_root)):
  all_txts=[]
  print('annotaion file : (%d / %d)'%(i,len(os.listdir(annotation_root))))
  session_file = pd.read_csv(os.path.join(annotation_root,annotation)) 
  # print(session_file) 
  segments=session_file['Segment ID']
  # print(segments)
  
  for i in range(1,len(segments)):
    #index starts from 1

    f_name=segments[i]
    sess_num=f_name.split('_')[0][-2:]
    sc_pro_num=f_name.split('_')[0]+'_'+f_name.split('_')[1]
    # print(sess_num,sc_pro_num)
    
  #  /content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/wav/Session01/Sess01_script01/Sess01_script01_F001.txt
    file_loc='/content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/wav/Session%s/%s/%s.txt'%(sess_num,sc_pro_num,f_name)
    # print(file_loc)
    if not os.path.isfile(file_loc):
      print("no file location :",file_loc)
      no_file+=1
    with open(file_loc) as f:
      lines = f.readlines()
      all_txts.append(lines)



  session_file.loc[(session_file['Total Evaluation'] == "fear"), 'Total Evaluation'] = 0  #공포 => 0
  session_file.loc[(session_file['Total Evaluation'] == "surprise"), 'Total Evaluation'] = 1  #놀람 => 1
  session_file.loc[(session_file['Total Evaluation'] == "angry"), 'Total Evaluation'] = 2  #분노 => 2
  session_file.loc[(session_file['Total Evaluation'] == "sad"), 'Total Evaluation'] = 3  #슬픔 => 3
  session_file.loc[(session_file['Total Evaluation'] == "neutral"), 'Total Evaluation'] = 4  #중립 => 4
  session_file.loc[(session_file['Total Evaluation'] == "happy"), 'Total Evaluation'] = 5  #행복 => 5
  session_file.loc[(session_file['Total Evaluation'] == "disgust"), 'Total Evaluation'] = 6  #혐오 => 6



  data_list=[]
  # print(len(all_txts))
  ## zip txts and labels,  session_file start from 1
  for q, label in zip(all_txts, session_file['Total Evaluation'][1:])  :
    if type(label) is not int :
      ambig_annotation+=1
      continue
    data = []
    data.append(str(q))
    data.append(str(label))
    data_list.append(data)
  print(data_list)
  allfile_datalist=allf.append(data_list)

# print("data_list length :",len(data_list))
# print("sample : ",data_list[0])
    
# print(len(all_txts))





annotaion file : (0 / 40)
[["['어 저 지그 지금 사람 친 거야? 지금 사람 친 거 맞지? 그치?\\n']", '1'], ["['b/ 몰라. o/ b/ 아 몰라 어떡해. o/\\n']", '0'], ["['아이 씨 그러니까 나 말렸어야지. 술 먹어서 운전 안 한다고 했잖아. b/\\n']", '0'], ["['아이 씨 괜히 운전해서 이씨.\\n']", '0'], ["['n/ 지섭씨. 일단 112에 신고하자.\\n']", '0'], ["['잠깐만. 잠깐만 기다려봐. 이 그러니 그러니까 그 좀 생각좀 o/ 해보자고 그. o/\\n']", '0'], ["['아 지금 생각할게 할게 뭐가 있어. b/ 지섭씨는 여기있어. 일단 내가 가서 보고 o/ 올테니까.\\n']", '0'], ["['o/ u/ 죽었으면 u/ 어떡하지? 나 수 술도 n/ 술도 마셨는데 u/ 신고를 해야되나? 아이 그니까 이게 u/ 나중에 더 일이 u/ 커질건데 그면 어떡하지. 그니까 내가 그니까 이거. b/\\n']", '0'], ["['이 야밤에 돌아다니다가 왜 내 차에 치이고 난리야 이씨. b/ \\n']", '0'], ["['지섭씨.\\n']", '2'], ["['아 아니 그게 말야. 아무것도 도로에 아무것도 없어.\\n']", '0'], ["['어? 아무것도 없다니? 그게 말이 돼? 아 틀림없이 사람을 쳤잖아. 너도 봤잖아. \\n']", '0'], ["['아 그래 근데 지금 없어. 죽었는지 살았는지 우리가 친 사람이 없단 말야.\\n']", '0'], ["['u/ 그 그럼 뭐지? 어? 뭐 뭐.\\n']", '0'], ["['o/ 아 l/ 아 그래 맞아. 고라니였나보다. 그치? 고라니였어. l/ 그니까 너랑 나랑 술을 마셔서 고라니를 이 사람으로 착각을 한 거야. 그지?\\n']", '0'], ["['o/ N/ 지섭씨 사람하고 고라니를 구분 못 할 정돈 (아니잖아)\\n']", '4'], ["['하 그러니까. o/ b/\\n']", '0'], ["['잠깐만. 잠깐만 우 

KeyboardInterrupt: ignored

In [104]:
a=[['a'],1]
b=[['b'],2]
a=a+b
a=a+b
print(a)

[['a'], 1, ['b'], 2, ['b'], 2]


In [77]:

# separate emotion evaluation 
# for i in range(1,len(segments)):
#   if i ==2:
#     break
#   instance=sess1_f[i:i+1]
  
#   emotions=[]
#   for col in sess1_f.columns:
    
#     # print(col)
#     if 'Eval' in col:
#       # print(col)
#       emotions.append(instance[col])
#     # print('emotions')
#   print(emotions)

##use total evaluation
sess1_f.loc[(sess1_f['Total Evaluation'] == "fear"), 'Total Evaluation'] = 0  #공포 => 0
sess1_f.loc[(sess1_f['Total Evaluation'] == "surprise"), 'Total Evaluation'] = 1  #놀람 => 1
sess1_f.loc[(sess1_f['Total Evaluation'] == "angry"), 'Total Evaluation'] = 2  #분노 => 2
sess1_f.loc[(sess1_f['Total Evaluation'] == "sad"), 'Total Evaluation'] = 3  #슬픔 => 3
sess1_f.loc[(sess1_f['Total Evaluation'] == "neutral"), 'Total Evaluation'] = 4  #중립 => 4
sess1_f.loc[(sess1_f['Total Evaluation'] == "happy"), 'Total Evaluation'] = 5  #행복 => 5
sess1_f.loc[(sess1_f['Total Evaluation'] == "disgust"), 'Total Evaluation'] = 6  #혐오 => 6



data_list=[]
# print(len(all_txts))
## zip txts and labels,  sess1_f start from 1
for q, label in zip(all_txts, sess1_f['Total Evaluation'][1:])  :
  # if label is not '0' or '1' or '2' or '3' or '4' or '5' or '6':
  #   print(label)
  if type(label) is not int :
    print(label)
    continue
  data = []
  data.append(str(q))
  data.append(str(label))
  data_list.append(data)
print("data_list length :",len(data_list))
print("sample : ",data_list[0])

fear;surprise
neutral;fear
neutral;happy
neutral;happy
neutral;surprise
neutral;happy
neutral;angry
neutral;angry
neutral;disgust
neutral;disgust
disgust;surprise
neutral;surprise
neutral;surprise
happy;surprise
neutral;happy
neutral;happy
neutral;happy
neutral;happy
neutral;disgust
neutral;surprise
neutral;surprise
neutral;happy
neutral;happy
data_list length : 458
sample :  ["['어 저 지그 지금 사람 친 거야? 지금 사람 친 거 맞지? 그치?\\n']", '1']


In [78]:
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)
print(len(dataset_train))
print(len(dataset_test))

343
115


In [79]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [80]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [81]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [84]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [86]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [87]:
##train
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-87-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/6 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.067678213119507 train acc 0.09375
epoch 1 train acc 0.1928215579710145


<ipython-input-87-480b6a139979>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/2 [00:00<?, ?it/s]

...

epoch 4 test acc 0.5186887254901961


  0%|          | 0/6 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 1.6179388761520386 train acc 0.46875
epoch 5 train acc 0.47554347826086957


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 5 test acc 0.5186887254901961
